## Sentiment Analysis using LSTM (Bi-Di)


In [5]:
import tensorflow as tf
import os
import numpy as np
import pandas as pd
# Import Tokenizer and pad_sequences
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

dataset = pd.read_csv('/Users/avaneeshkumar/Downloads/Docker/DL/NLP-using-tf/NLP/combined_data.csv')

# Extract out sentences and labels
sentences = dataset['text'].tolist()
labels = dataset['sentiment'].tolist()

In [3]:


vocab_size = 1000
tokenizer = tfds.features.text.SubwordTextEncoder.build_from_corpus(sentences, vocab_size, max_subword_length=5)

# How big is the vocab size?
print("Vocab size is ", tokenizer.vocab_size)

Vocab size is  999


In [4]:
# Check that the tokenizer works appropriately
num = 5
print(sentences[num])
encoded = tokenizer.encode(sentences[num])
print(encoded)


# Separately print out each subword, decoded
for i in encoded:
    print(tokenizer.decode([i]))

I have to jiggle the plug to get it to line up right to get decent volume.
[4, 31, 6, 849, 162, 450, 12, 1, 600, 438, 775, 6, 175, 14, 6, 55, 213, 159, 474, 775, 6, 175, 614, 380, 295, 148, 72, 789]
I 
have 
to 
j
ig
gl
e 
the 
pl
ug
 
to 
get 
it 
to 
li
ne 
up 
right
 
to 
get 
dec
ent 
vo
lu
me
.


### Replace sentence data with encoded subwords

In [5]:
for i, sentence in enumerate(sentences):
    sentences[i] = tokenizer.encode(sentence)

### Final Preporcessing 

In [6]:
import numpy as np

max_length = 50
trunc_type='post'
padding_type='post'

# Pad all sequences
sequences_padded = pad_sequences(sentences, maxlen=max_length, 
                                 padding=padding_type, truncating=trunc_type)

# Separate out the sentences and labels into training and test sets
training_size = int(len(sentences) * 0.8)

training_sequences = sequences_padded[0:training_size]
testing_sequences = sequences_padded[training_size:]
training_labels = labels[0:training_size]
testing_labels = labels[training_size:]

# Make labels into numpy arrays for use with the network later
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [8]:
embedding_dim = 16

# Define the model
model_bidi_lstm = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim)), 
    tf.keras.layers.Dense(6, activation='relu'), 
    tf.keras.layers.Dense(1, activation='sigmoid')
])


In [10]:
# training
num_epochs = 20
model_bidi_lstm.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

history = model_bidi_lstm.fit(training_sequences, training_labels_final, epochs=num_epochs, validation_data=(testing_sequences, testing_labels_final))


Train on 1593 samples, validate on 399 samples
Epoch 1/20
1593/1593 [==============================] - 17s 10ms/sample - loss: 0.6928 - accuracy: 0.5097 - val_loss: 0.6962 - val_accuracy: 0.4110
Epoch 2/20
1593/1593 [==============================] - 6s 4ms/sample - loss: 0.6881 - accuracy: 0.5499 - val_loss: 0.6776 - val_accuracy: 0.7018
Epoch 3/20
1593/1593 [==============================] - 6s 4ms/sample - loss: 0.6167 - accuracy: 0.7313 - val_loss: 0.5728 - val_accuracy: 0.7494
Epoch 4/20
1593/1593 [==============================] - 6s 4ms/sample - loss: 0.4655 - accuracy: 0.8110 - val_loss: 0.5356 - val_accuracy: 0.7619
Epoch 5/20
1593/1593 [==============================] - 6s 4ms/sample - loss: 0.3525 - accuracy: 0.8675 - val_loss: 0.4867 - val_accuracy: 0.7794
Epoch 6/20
1593/1593 [==============================] - 6s 4ms/sample - loss: 0.2692 - accuracy: 0.9083 - val_loss: 0.5329 - val_accuracy: 0.7594
Epoch 7/20
1593/1593 [==============================] - 7s 5ms/sample - los

In [ ]:
### PLotting the learning rate

In [ ]:
import matplotlib.pyplot as plt

def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_'+string])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string, 'val_'+string])
    plt.show()

plot_graphs(history, "accuracy")
plot_graphs(history, "loss")

In [2]:
my_reviews =["lovely", "dreadful", "stay away",
             "everything was hot exactly as I wanted",
             "everything was not exactly as I wanted",
             "they gave us free chocolate cake",
             "I've never eaten anything so spicy in my life, my throat burned for hours",
             "for a phone that is as expensive as this one I expect it to be much easier to use than this thing is",
             "we left there very full for a low price so I'd say you just can't go wrong at this place",
             "that place does not have quality meals and it isn't a good place to go for dinner",
             ]

In [4]:
print("===================================\n", "With a single bidirectional LSTM:\n", "===================================")
predict_review(model_bidi_lstm, my_reviews, show_padded_sequence=False)